<a href="https://colab.research.google.com/github/iloveleejunghyun/CMPE_HW2_DigitsRecognition/blob/main/258_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip3 install -U scikit-learn==0.24.1

Requirement already up-to-date: scikit-learn==0.24.1 in /usr/local/lib/python3.7/dist-packages (0.24.1)


In [7]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from PIL import Image
import io
import cv2
from google.colab.patches import cv2_imshow
import joblib
from skimage.feature import hog


In [17]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
!wget https://github.com/abhi9716/handwritten-MNIST-digit-recognition/raw/master/photo_1.jpg
!wget https://github.com/iloveleejunghyun/CMPE_HW2_DigitsRecognition/raw/main/digits_cls1.pkl

--2021-03-20 12:04:45--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml.2’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.02s   

2021-03-20 12:04:45 (58.3 MB/s) - ‘haarcascade_frontalface_default.xml.2’ saved [930127/930127]

--2021-03-20 12:04:45--  https://github.com/abhi9716/handwritten-MNIST-digit-recognition/raw/master/photo_1.jpg
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/abh

In [18]:
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }


    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

def recognize(im):
  # Read the input image 
  # im = cv2.imread("photo_1.jpg")
  # Convert to grayscale and apply Gaussian filtering
  im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)
  # Threshold the image
  ret, im_th = cv2.threshold(im_gray, 70, 255, cv2.THRESH_BINARY_INV)

  # Find contours in the image
  ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Get rectangles contains each contour
  rects = [cv2.boundingRect(ctr) for ctr in ctrs]

  for rect in rects:
    # Draw the rectangles
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
    # Make the rectangular region around the digit
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
    # Resize the image
    try:
      roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
      roi = cv2.dilate(roi, (3, 3))
      # Calculate the HOG features
      roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1))
      roi_hog_fd = pp.transform(np.array([roi_hog_fd], 'float64'))
      # Pridict digit
      nbr = clf.predict(roi_hog_fd)
      cv2.putText(im, str(int(nbr[0])), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
    except BaseException:
      # print("size problem!")
      pass
  return im

VideoCapture()
eval_js('create()')

# Load the classifier
clf,pp = joblib.load("digits_cls1.pkl")

# Create the haar cascade
face_detector = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')

while True:
  # Capture frame-by-frame
  byte = eval_js('capture()')
  im = byte2image(byte)
  im = recognize(im)
  # # Our operations on the frame come here
  # gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  # edges = cv2.Canny(gray,100,200)

  # # Detect faces in the image
  # faces = face_detector.detectMultiScale(gray, 1.1, 4)

  # # Draw a rectangle around the faces
  # for(x, y, w, h) in faces:
  #   cv2.rectangle(im, (x,y), (x+w,y+h), (255,0,0), 2)

  
  # Display the resulting frame
  #下面这一行用来显示加数字的
  eval_js('showimg("{}")'.format(image2byte(im)))
  # eval_js('showimg("{}")'.format(image2byte(edges)))



<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored